# 1. Data Analysis & Preparation

We begin by loading the raw datasets and merging them to create a unified view of "Sales vs. Logistics Cost".

In [1]:
import pandas as pd 
import os

try:
    # The ".." means go up one level to the main folder
    items = pd.read_csv('Data/olist_order_items_dataset.csv')
    products = pd.read_csv('Data/olist_products_dataset.csv')
    payments = pd.read_csv('Data/olist_order_payments_dataset.csv')
    
    if items.empty or products.empty or payments.empty:
        raise ValueError("One or more datasets are empty. Check the content of the CSV files.")
    
    print("Datasets loaded successfully.")
except FileNotFoundError:
    print("Error: CSV files not found. Check your 'data' folder location.")
except ValueError as e:
    print(e)

Datasets loaded successfully.


# Dataset Overview

In [2]:
# Check shapes and missing values
for name, df in {'Items': items, 'Products': products, 'Payments': payments}.items():
    print(f"{name} Dataset: {df.shape[0]} rows, {df.shape[1]} columns")
    print(f"Missing values in {name} dataset:\n{df.isna().sum()}\n")

Items Dataset: 112650 rows, 7 columns
Missing values in Items dataset:
order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64

Products Dataset: 32951 rows, 9 columns
Missing values in Products dataset:
product_id                      0
product_category_name         610
product_name_lenght           610
product_description_lenght    610
product_photos_qty            610
product_weight_g                2
product_length_cm               2
product_height_cm               2
product_width_cm                2
dtype: int64

Payments Dataset: 103886 rows, 5 columns
Missing values in Payments dataset:
order_id                0
payment_sequential      0
payment_type            0
payment_installments    0
payment_value           0
dtype: int64



# Clean & Merge

In [3]:
# --- MERGE & CLEAN ---
## 1. Filter for valid paid orders
if 'order_id' in payments.columns and 'order_id' in items.columns:
    valid_orders = payments['order_id'].unique()
    original_rows = items.shape[0]
    items = items[items['order_id'].isin(valid_orders)]
    print(f"Filtered valid orders. Rows reduced from {original_rows} to {items.shape[0]}.")
else:
    print("Error: 'order_id' column missing in one of the datasets.")

# 2. Merge Items with Products (Link Price to Weight)
if 'product_id' in items.columns and 'product_id' in products.columns:
    df = pd.merge(items, products, on='product_id', how='inner')
    print(f"Merged dataset created with {df.shape[0]} rows and {df.shape[1]} columns.")
else:
    print("Error: 'product_id' column missing in one of the datasets.")

# 3. Drop Errors (Missing values or 0s)
before_cleaning = df.shape[0]
df = df.dropna(subset=['product_category_name', 'product_weight_g', 'price'])
df = df[(df['product_weight_g'] > 0) & (df['price'] > 0)]
print(f"Cleaned dataset. Rows reduced from {before_cleaning} to {df.shape[0]}.")

Filtered valid orders. Rows reduced from 112650 to 112647.
Merged dataset created with 112647 rows and 15 columns.
Cleaned dataset. Rows reduced from 112647 to 111035.


In [4]:
# 4. Outlier Handling
# We cap items at 30kg. Items heavier than this usually require special freight 
# logic, so we exclude them to focus on standard warehouse capacity.
df_clean = df[df['product_weight_g'] <= 30000].copy()
print(f"Original Rows: {len(df)}")
print(f"Cleaned Rows:  {len(df_clean)}")

Original Rows: 111035
Cleaned Rows:  111032


# Aggregate & Save Parameters

In [5]:
# Aggregate
lp_data = df_clean.groupby('product_category_name').agg({
    'price': 'mean', # Pi: average revenue
    'product_weight_g': 'mean', # wi : average  weight
    'order_item_id': 'count' # Di : total historical Demand
}).reset_index()
# rename columns
lp_data.columns = ['Category', 'P_i', 'W_i', 'D_i']
lp_data.to_csv('Data/lp_parameters.csv', index=False)
with open('Data/capacity.txt', 'w') as f:
    f.write(str(df_clean['product_weight_g'].sum()))
print("Success! Created 'data/lp_parameters.csv' and 'data/capacity.txt'")

Success! Created 'data/lp_parameters.csv' and 'data/capacity.txt'
